In [3]:
import requests
from elasticsearch import Elasticsearch

In [6]:
#fetch docs
docs_url = 'https://raw.githubusercontent.com/anjelammcgraw/LLM-Zoomcamp/main/01%20intro/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [7]:
#prep docs
documents = []
for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
#connect to Elasticsearch
es = Elasticsearch("http://localhost:9200")

In [9]:
#index settings/mappings
index_name = "faq_index"
index_settings = {
    "mappings": {
        "properties": {
            "course": {"type": "keyword"},
            "question": {"type": "text"},
            "text": {"type": "text"}
        }
    }
}


In [10]:
#create index
es.indices.create(index=index_name, body=index_settings, ignore=400)

/tmp/ipykernel_69427/2880972178.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_name, body=index_settings, ignore=400)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'faq_index'})

In [11]:
#index docs
for i, doc in enumerate(documents):
    es.index(index=index_name, id=i, body=doc)

In [13]:
#searching
query = {
    "query": {
        "multi_match": {
            "query": "How do I execute a command in a running docker container?",
            "fields": ["question^4", "text"],
            "type": "best_fields"
        }
    }
}

response = es.search(index=index_name, body=query)
top_score = response['hits']['hits'][0]['_score']

print(top_score)

84.050095


In [15]:
#filtering
query = {
    "query": {
        "bool": {
            "must": [
                {"multi_match": {
                    "query": "How do I execute a command in a running docker container?",
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }},
                {"term": {"course": "machine-learning-zoomcamp"}}
            ]
        }
    },
    "size": 3
}

response = es.search(index=index_name, body=query)
third_question = response['hits']['hits'][2]['_source']['question']

print(third_question)

How do I copy files from a different folder into docker container’s working directory?


In [16]:
#building prompt
context_template = """
Q: {question}
A: {text}
""".strip()

# Construct the context
context = "\n\n".join([
    context_template.format(question=hit['_source']['question'], text=hit['_source']['text'])
    for hit in response['hits']['hits']
])

question = "How do I execute a command in a running docker container?"
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=question, context=context)

print(len(prompt))

1462


In [18]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.5 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 17.9 MB/s eta 0:00:00m eta 0:00:01


In [20]:
#tokens
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4")
tokens = encoding.encode(prompt)

print(len(tokens))

325


In [25]:
#bonus: calucating costs
from openai import OpenAI

In [26]:
client = OpenAI()

In [30]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

#calculate costs
input_tokens = 150
output_tokens = 250
cost_per_request = (input_tokens / 1000 * 0.005) + (output_tokens / 1000 * 0.015)
total_cost = cost_per_request * 1000

print(total_cost)

4.5
